# <font color="#49699E" size=40>Processing Structured Data</font>

# LEARNING OBJECTIVES
# LEARNING MATERIALS


# INTRODUCTION


## Imports

In [ ]:
import pandas as pd
pd.set_option("display.notebook_repr_html", False)

## PRACTICAL PANDAS: FIRST STEPS

### Getting Data into Pandas


In [ ]:
df = pd.read_csv('../data/vdem/V-Dem-CY-Full+Others-v10.csv', low_memory=False)

In [ ]:
df.shape

### What Do You Need? Selecting Columns


In [ ]:
subset_vars = ['country_name', 'country_text_id', 'e_regiongeo', 'year', 'v2x_polyarchy', 'v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem', 'v2smprivex', 'v2smpolsoc', 'v2caviol', 'e_boix_regime']
sdf = df[subset_vars]
sdf.shape

In [ ]:
list(sdf.columns)

#### What's in Your dataframe?


In [ ]:
sdf.info()

In [ ]:
sdf[['e_regiongeo', 'year', 'v2x_polyarchy']].describe()

### Heads, Tails, and Samples


In [ ]:
sdf[['country_name', 'year', 'v2x_libdem']].head()

In [ ]:
sdf[['country_name', 'year', 'v2x_libdem']].tail(3)

In [ ]:
sdf[['country_name', 'year', 'v2x_libdem']].sample(15)

### What Do You Need? Filtering Rows

In [ ]:
rowfilter = sdf['year'] >= 1900
fsdf = sdf[rowfilter].copy()
fsdf.info()

In [ ]:
alternate_fsdf = sdf.query('year >= 1900').copy()
alternate_fsdf.info()

### Writing Data to Disk


In [ ]:
fsdf.to_csv('../data/vdem/filtered_subset.csv', index=False)

## UNDERSTANDING PANDAS DATA STRUCTURES


### The `Series`


In [ ]:
fsdf['v2x_delibdem'].sample(15, random_state = 42)

In [ ]:
fsdf.index

#### Accessing a Specific Row by its Index


In [ ]:
fsdf.loc[202]

In [ ]:
fsdf['v2x_delibdem'].loc[202]

In [ ]:
fsdf['v2x_delibdem'].loc[20000]

In [ ]:
fsdf.iloc[18786]

In [ ]:
fsdf.reset_index(inplace = True, drop = True)
fsdf.loc[18786]

### Dataframes


In [ ]:
deliberative = fsdf['v2x_delibdem']

In [ ]:
fsdf['21 Century'] = fsdf['year'] >= 2000
fsdf[['21 Century']].value_counts()

In [ ]:
fsdf['missing_political_violence_data'] = fsdf['v2caviol'].isna()
fsdf['missing_political_violence_data'].value_counts()

### Missing Data


In [ ]:
import numpy as np

type(np.nan)

In [ ]:
n = np.nan 
n == n

In [ ]:
if np.nan is None: 
    print('NaN is None') 
if np.nan: 
    print('NaN evaluates to True in control flow') 
if np.isnan(np.nan): 
    print('NaN is considered a NaN value in NumPy')

In [ ]:
total = len(fsdf['v2caviol']) 
count = fsdf['v2caviol'].count() 
print(f'Total: {total}') 
print(f'Count: {count}') 
print(f'Diff: {total-count}')

In [ ]:
nans = fsdf['v2caviol'].isna().sum() 
print(' NaNs: {}'.format(nans))

## AGGREGATION & GROUPED OPERATIONS


In [ ]:
grouped = fsdf.groupby('e_regiongeo')

In [ ]:
south_east_asia = grouped.get_group(13)
south_east_asia[['country_name', 'year', 'e_boix_regime']].head()

In [ ]:
poly = grouped['v2x_polyarchy'].median()
poly.head()

In [ ]:
regions = {
    1:'Western Europe',
    2:'Northern Europe',
    3:'Southern Europe',
    4:'Eastern Europe',
    5:'Northern Africa',
    6:'Western Africa',
    7:'Middle Africa',
    8:'Eastern Africa',
    9:'Southern Africa',
    10:'Western Asia',
    11:'Central Asia',
    12:'East Asia',
    13:'South-East Asia',
    14:'South Asia',
    15:'Oceania', # (including Australia and the Pacific)
    16:'North America',
    17:'Central America',
    18:'South America',
    19:'Caribbean' # (including Belize Cuba Haiti Dominican Republic)
}

In [ ]:
fsdf['Region'] = fsdf['e_regiongeo'].map(regions)

In [ ]:
grouped = fsdf.groupby(['Region', 'year'])
poly = grouped['v2x_polyarchy'].median()
poly.reset_index()
pd.DataFrame(poly).reset_index().sample(10)

In [ ]:
grouped.size().sort_values(ascending=False)

In [ ]:
with_agg = grouped['v2x_polyarchy'].agg([min, np.median, 'max', 'count'])
with_agg.reset_index().sample(10)

## WORKING WITH TIME SERIES DATA


In [ ]:
"Monday Mar 2, 1999" > "Friday Feb 21, 2020"

In [ ]:
import os
data_dir = os.listdir("../data/russian-troll-tweets/")

files = [f for f in data_dir if 'csv' in f]

tweets_df = pd.concat((pd.read_csv(
    f'{"../data/russian-troll-tweets/"}/{f}', 
    encoding='utf-8', low_memory=False) for f in files), ignore_index=True)

tweets_df.info()

In [ ]:
tweets_df['dt_publish_date'] = pd.to_datetime(tweets_df['publish_date'])
tweets_df['dt_harvested_date'] = pd.to_datetime(tweets_df['harvested_date'])

In [ ]:
tweets_df[['author', 'content', 'publish_date']].sample(5)

In [ ]:
tweets_df['dt_publish_date'].dt.month

In [ ]:
sorted_df = tweets_df.sort_values(['dt_publish_date'])

In [ ]:
tweets_df['days_until_harvest'] = tweets_df['dt_harvested_date'] - tweets_df['dt_publish_date']
tweets_df['days_until_harvest'].sample(10) 

In [ ]:
tweets_df['Year'] = tweets_df['dt_publish_date'].dt.year
tweets_df['Month'] = tweets_df['dt_publish_date'].dt.month
tweets_df['Day'] = tweets_df['dt_publish_date'].dt.day

In [ ]:
tweets_df = tweets_df.set_index('dt_publish_date')

In [ ]:
grouped_cal_day = tweets_df.resample('D').size()
grouped_cal_day

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from dcss.plotting import custom_seaborn
custom_seaborn()

In [ ]:
sns.lineplot(data=grouped_cal_day, color='#32363A')
sns.despine()
plt.show()

In [ ]:
weekly = tweets_df.resample('W').size()
weekly.head()

In [ ]:
ax = sns.lineplot(data=weekly, color='#32363A')
ax.set_xlabel('\nWeekly observations')
ax.set_ylabel('Number of Tweets\n')
sns.despine()
plt.show()

## COMBINING DATAFRAMES


In [ ]:
full_df =  pd.read_csv("../data/vdem/filtered_subset.csv")
df_australia = full_df.query("country_name == 'Australia'")
len(df_australia)

In [ ]:
df_sa = full_df.query("country_name == 'South Africa'")
len(df_sa)

In [ ]:
concatenated = pd.concat([df_australia, df_sa], axis=1)
len(concatenated)

### Merging


In [ ]:
freedom_df = pd.read_csv( "../data/freedom_house/internet_freedoms_2020.csv")

In [ ]:
fsdf.columns

In [ ]:
freedom_df.columns

In [ ]:
merged = pd.merge(fsdf, freedom_df, how='inner', left_on='country_name', right_on='Country')
print('merged has {} rows and {} columns'.format(len(merged), len(merged.columns)))

In [ ]:
len(fsdf) + len(freedom_df)

In [ ]:
fsdf_set = set(fsdf['country_name'].tolist())
freedom_set = set(freedom_df['Country'].tolist())

unmatched = fsdf_set ^ freedom_set

print('Total countries: ' + str(len(fsdf_set) + len(freedom_set)))
print('Unmatched countries: ' + str(len(unmatched)))

In [ ]:
fsdf_missing = list(fsdf_set & unmatched)
fsdf_missing.sort()
freedom_missing = list(freedom_set & unmatched)
freedom_missing.sort()
pd.DataFrame({'VDEM': pd.Series(fsdf_missing), 'Freedom': pd.Series(freedom_missing)})

### Record Linkage


# CONCLUSION
## Key Points 
